# **Resolución de la ecuación del calor bidimensional aplicada a una barra de largo L y ancho L**

## **Importación de librerías y preparación de entorno**

In [ ]:
# %env DDE_BACKEND=tensorflow.compat.v1
%env DDE_BACKEND = tensorflow

In [ ]:
import numpy as np
import deepxde as dde
from deepxde.backend import tf
from modulePINN import Geometry, IC_BC, loadData, PINN, Ploter, Makegrid

## **Definición de parámetros geométricos del problema**

In [ ]:
params_placa = [0, 1, [0, 0], [1, 1]]
placa = Geometry("placa")
geom_placa, timedomain_placa, geomtime_placa = placa.geometry_domain(params_placa)

In [ ]:
icbc = IC_BC(geomtime_placa)
ic_placa, bc_placa = icbc.problem_conditions(0.2, f = lambda x: 2*np.sin(1 * np.pi * x[:, 0:1] / 1))

## **Datos sintéticos y entrenamiento**

In [ ]:
def pde2d(x, u):


  a = 0.4

  du_t = dde.grad.jacobian(u, x, i=0, j=1)
  du_xx = dde.grad.hessian(u, x, i=0, j=0)
  du_yy = dde.grad.hessian(u, x, i=1, j=1)

  return du_t - a * (du_xx + du_yy)

In [ ]:
data_placa = loadData(pde2d, geomtime_placa, ic_placa, bc_placa).get_data(2050, 1500, 200, 250)

In [ ]:
plots = Ploter((8,6)) # Construimos el objeto graficador para la barra
plots.collocation(data_placa) # Graficamos los puntos de colocación (train)

### **Entrenamiento**

In [ ]:
pinn_placa = PINN(n_input=3, n_hidden=30, n_output=1, activation="tanh", mmm = "Glorot normal")
model1, losshistory1, train_state1 = pinn_placa.train_model(data_placa)

## **Visualización de resultados**

In [ ]:
plots.train_plot(losshistory1, train_state1)

In [ ]:
n=100
k = 0
grid = Makegrid(mode="3d")
grid_terms_placa = grid.grid_3D(n, k, params_placa, model1)

In [ ]:
plots.surface_plot(grid_terms_placa, mode="placa")

In [ ]:
plots.heatmap_plot(grid_terms_placa, mode="placa")

## **Animación**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# --- 1. Parámetros del dominio y malla de evaluación ---
Lx = 1.0       # corresponde al Rectangle([0,0],[1,1])
Ly = 1.0
T_end = 1.0    # dominio temporal [0, 1]
nx, ny, nt = 100, 100, 100  # resolución espacial y temporal

x_vals = np.linspace(0, Lx, nx)
y_vals = np.linspace(0, Ly, ny)
t_list = np.linspace(0, T_end, nt)

# Crear malla (x, y, t)
X, Y, T = np.meshgrid(x_vals, y_vals, t_list, indexing='ij')  # forma (nx, ny, nt)
XT = np.vstack([X.ravel(), Y.ravel(), T.ravel()]).T         # array shape (nx*ny*nt, 3)

# --- 2. Predicción con el modelo DeepXDE ---
u_pred = model1.predict(XT)[:, 0]  # vector (nx*ny*nt,)
U = u_pred.reshape((nx, ny, nt))  # tensores organizados
U = np.transpose(U, (2, 0, 1))    # ahora U[k], k=0,…,nt−1

# --- 3. Establecer figura e imagen inicial para animación ---
fig, ax = plt.subplots(figsize=(6, 5))
vmin, vmax = U.min(), U.max()

im = ax.imshow(
    U[0],
    origin="lower",
    cmap="jet",
    vmin=0,
    vmax=1,
    animated=True
)
cbar = fig.colorbar(im, ax=ax, label=r"$u(x,y,t)$")
title = ax.set_title(f"t = {t_list[0]:.3f}")

ax.set_xlabel("x")
ax.set_ylabel("y")

# --- 4. Funciones para init y animación ---
def init():
    im.set_data(U[0])
    title.set_text(f"t = {t_list[0]:.3f}")
    return im, title

def animate(k):
    im.set_data(U[k])
    title.set_text(f"t = {t_list[k]:.3f}")
    return im, title

# --- 5. Construcción de la animación ---
interval_ms = 10*max(20, int(1000 * (t_list[1] - t_list[0])))
anim = animation.FuncAnimation(
    fig,
    animate,
    init_func=init,
    frames=nt,
    interval=interval_ms,
    blit=True
)

# plt.tight_layout()

# Para exportar como archivo:
# anim.save("heat2d_evolution.mp4", writer="ffmpeg", dpi=150)
# anim.save("heat2d_evolution.gif", writer='pillow')

plt.close(fig)
HTML(anim.to_jshtml())